# Creación de un mapa de contaminación

Se pretende cargar la posición de las estaciones de medición de contaminantes junto con sus contaminantes y realizar una interpolación de estos contaminantes en todo un mallado entre las estaciones conocidas.

In [1]:
import numpy as np
import pandas as pd
import re
from scipy.interpolate import SmoothBivariateSpline
import matplotlib.pyplot as plt 
import pyarrow #uninstall libboost & conda install pyarrow=0.13.* -c conda-forge

In [2]:
def dms2dd(input):
    "Esta función tranforma un punto expresado en coordenadas geográficas a coordenadas UTM"
    degrees,minutes,seconds,direction=re.split('[º\´"]+',str(input))
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

Se leeran las coordenadas de las estaciones y se filtrarán las que son de nuestro interés

In [3]:
posicion_estaciones=pd.read_excel('./data/EstacionesLocalizacion.xlsx') #Se carga el fichero

Se filtra el fichero de estaciones a la estaciones de nuestro estudio

In [4]:
Estacion=[4,8,11,35,38,39,47,48,49,50]
posicion_estaciones=posicion_estaciones[posicion_estaciones['Estacion'].isin(Estacion)]
posicion_estaciones.set_index('Estacion',inplace=True)
posicion_estaciones

,Columna1,Latitud,Longitud
Estacion,,,
47,Mendez Alavaro,"40º23´53.17""N","3º41´12.57""O"
11,Av Ramon y Cajal,"40º27´5.29""N","3º40´38.5""O"
8,Escuelas Aguirre,"40º25´17.63""N","3º40´56.22""O"
49,Retiro,"40º24´52""N","3º40´57.3""O"
50,Plaza Castilla,"40º27´56.1""N","3º41´19.48""O"
48,Paseo Castellana,"40º26´23.61""N","3º41´25.34""O"
35,Plaza del carmen,"40º25´9.15""N","3º42´11.4""O"
38,Cuatro Caminos,"40º26´43.97""N","3º42´25.64""O"
39,Barrio del pilar,"40º28´41.64""N","3º42´41.53""O"


Se convertirán los valores de Latitud y Longitud en coordenadas decimales

In [5]:
posicion_estaciones[['Latitud','Longitud']]=posicion_estaciones[['Latitud','Longitud']].applymap(lambda x: dms2dd(x))

In [6]:
posicion_estaciones.sort_values(['Latitud','Longitud'])

,Columna1,Latitud,Longitud
Estacion,,,
47,Mendez Alavaro,40.398103,3.686825
49,Retiro,40.414444,3.682583
35,Plaza del carmen,40.419208,3.703167
8,Escuelas Aguirre,40.421564,3.682283
4,Plaza de España,40.423883,3.712197
48,Paseo Castellana,40.439892,3.690372
38,Cuatro Caminos,40.445547,3.707122
11,Av Ramon y Cajal,40.451469,3.677361
50,Plaza Castilla,40.465583,3.688744


In [7]:
sortLatitud=posicion_estaciones['Latitud'].sort_values()
sortLongitud=posicion_estaciones['Longitud'].sort_values()

Se desea obtener una matriz de puntos sobre la que realizar una interpolación. La interpolación se realizará sobre los puntos conocidos, para obtener una matriz de valores contaminantes, que representará la polución en cada espacio geografico de Madrid capital

In [8]:
Latitudes=[]
Longitudes=[]
for i in range(0,(len(sortLatitud))):
    if i==9:
        lastLat=np.round(sortLatitud.iloc[i],decimals=6)
        lastLon=np.round(sortLongitud.iloc[i],decimals=6)
        Latitudes.append(lastLat)
        Longitudes.append(lastLon)
    else:
        inicioLat=sortLatitud.iloc[i]
        inicioLon=sortLongitud.iloc[i]
        finLat=sortLatitud.iloc[i+1]
        finLon=sortLongitud.iloc[i+1]
        betweenLat=np.round(np.arange(inicioLat,finLat,0.001),decimals=6)
        betweenLon=np.round(np.arange(inicioLon,finLon,0.001),decimals=6)
        Latitudes.extend(betweenLat)
        Longitudes.extend(betweenLon)

In [9]:
datos_estaciones=pd.read_csv('./data/NO2_estaciones.csv')
datos_estaciones.head()

,FECHA,Estacion4,Estacion8,Estacion11,Estacion35,Estacion38,Estacion39,Estacion47,Estacion48,Estacion49,Estacion50
0,2015-01-01 01:00:00,83.0,168.0,93.0,109.0,102.0,104.0,112.0,108.0,136.0,127.0
1,2015-01-01 02:00:00,77.0,203.0,224.0,133.0,183.0,168.0,120.0,114.0,111.0,170.0
2,2015-01-01 03:00:00,86.0,220.0,152.0,126.0,193.0,203.0,109.0,141.0,109.0,118.0
3,2015-01-01 04:00:00,65.0,169.0,151.0,112.0,122.0,147.0,87.0,145.0,99.0,126.0
4,2015-01-01 05:00:00,53.0,154.0,106.0,109.0,97.0,159.0,76.0,142.0,103.0,140.0


In [10]:
fechas=list(datos_estaciones.FECHA)

In [11]:
fechas2015=[]
fechas2016=[]
fechas2017=[]
fechas2018=[]
fechas2019=[]
fechas2020=[]
for date in fechas:
    if re.findall('2015', date):
        fechas2015.append(date)
    elif re.findall('2016', date):
        fechas2016.append(date)
    elif re.findall('2017', date):
        fechas2017.append(date)
    elif re.findall('2018', date):
        fechas2018.append(date)
    elif re.findall('2019', date):
        fechas2019.append(date)
    else:
        fechas2020.append(date)

In [12]:
ix1 = pd.MultiIndex.from_product([fechas,Latitudes])

In [13]:
mapa = pd.DataFrame(index=ix1, columns=Longitudes, dtype=np.float32)

In [ ]:
for ano in ['2016','2017','2018','2019','2020']:
    variable='fechas'+ano
    fechas=eval(variable)
    for date in fechas:
        print(date)
        x=[]
        y=[]
        Z=[]
        for i in Estacion:
            lat=posicion_estaciones.loc[i,'Latitud'].round(decimals=6)
            #print(lat)
            lon=posicion_estaciones.loc[i,'Longitud'].round(decimals=6)
            #print(lon)
            Est='Estacion'+str(i)
            sust=float(datos_estaciones[datos_estaciones['FECHA']==date][Est])
            #mapa.loc[(date,lat),lon]=sust
            x.append(lon)
            y.append(lat)
            Z.append(sust)
        #interp_spline=interpolate.interp2d(y,x,Z,kind='cubic')
        interp_spline = SmoothBivariateSpline(y,x,Z,kx=2,ky=2)
        mapa.loc[(date),:] = interp_spline(Latitudes,Longitudes)
mapa_interp=mapa.stack().reset_index()
filename='mapa_interpolado'+ano
mapa_interp.to_feather(filename)

2016-01-01 00:00:00


/anaconda3/lib/python3.6/site-packages/scipy/interpolate/fitpack2.py:1057: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)


2016-01-01 01:00:00
2016-01-01 02:00:00
2016-01-01 03:00:00
2016-01-01 04:00:00
2016-01-01 05:00:00
2016-01-01 06:00:00
2016-01-01 07:00:00
2016-01-01 08:00:00
2016-01-01 09:00:00
2016-01-01 10:00:00
2016-01-01 11:00:00
2016-01-01 12:00:00
2016-01-01 13:00:00
2016-01-01 14:00:00
2016-01-01 15:00:00
2016-01-01 16:00:00
2016-01-01 17:00:00
2016-01-01 18:00:00
2016-01-01 19:00:00
2016-01-01 20:00:00
2016-01-01 21:00:00
2016-01-01 22:00:00
2016-01-01 23:00:00
2016-01-02 00:00:00
2016-01-02 01:00:00
2016-01-02 02:00:00
2016-01-02 03:00:00
2016-01-02 04:00:00
2016-01-02 05:00:00
2016-01-02 06:00:00
2016-01-02 07:00:00
2016-01-02 08:00:00
2016-01-02 09:00:00
2016-01-02 10:00:00
2016-01-02 11:00:00
2016-01-02 12:00:00
2016-01-02 13:00:00
2016-01-02 14:00:00
2016-01-02 15:00:00
2016-01-02 16:00:00
2016-01-02 17:00:00
2016-01-02 18:00:00
2016-01-02 19:00:00
2016-01-02 20:00:00
2016-01-02 21:00:00
2016-01-02 22:00:00
2016-01-02 23:00:00
2016-01-03 00:00:00
2016-01-03 01:00:00
2016-01-03 02:00:00


In [ ]:
ano='2020'
filename='mapa_interpolado'+ano
filename

In [ ]:
mapa.loc[(edad),:]

In [ ]:
plt.matshow(mapa.loc[(date),:])
plt.show()

Para poder pasar los datos generados a una versión ordenada, que según Hadley Wickham, deberá realizar un "melt"

In [ ]:
mapa_interp=mapa.stack().reset_index()

In [ ]:
ano='2020'
filename='mapa_interpolado'+ano
posicion_estaciones.to_feather(filename)

In [ ]:
ano='2020'
variable='fechas'+ano
fechas=eval(variable)

In [ ]:
! which python